In [55]:
from fastapi import FastAPI, APIRouter, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import os
from dotenv import load_dotenv
from supabase import create_client, Client
import pandas as pd

load_dotenv()
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

async def staff_skill(staff_id: int):
    staff_skill = pd.DataFrame.from_dict(supabase.from_('staff_skill').select('skill(*)').eq('staff_id', staff_id).execute().data)['skill']
    return staff_skill.to_list()

async def staff_role_skill(staff_id: int, role_id: int):
    # Get the list of skill_id for the staff and role
    staff_skills = pd.DataFrame.from_dict(await staff_skill(staff_id))
    role_skills = pd.DataFrame.from_records(pd.DataFrame.from_dict(supabase.from_('role_skill').select('skill(*)').eq('role_id', role_id).execute().data)['skill'])

    # Add a 'qualified' field to the staff_skill list
    role_skills['qualified'] = role_skills['skill_id'].isin(staff_skills['skill_id'])

    # Sort the staff_skill list based on the 'qualified' field
    skill = role_skills.sort_values(by=['qualified', 'skill_name'], ascending=[False, True])
    match_percentage = skill['qualified'].mean()*100
    skill = skill.to_dict('records')

    staff_role_skill = {
        'match_percentage': match_percentage,
        'skill': skill
    }

    return staff_role_skill

role_id=2

# Get the list of skill_id for the staff and role
application = pd.DataFrame.from_records(supabase.table('application').select('*, staff(*)').eq('role_id', role_id).execute().data)
# pd.DataFrame.from_records(application.staff).staff_id.apply(lambda x: staff_skill(x))

tes = []
for i in application.staff:
    tes.append(await staff_role_skill(i['staff_id'], role_id))

2023-10-06 16:44:31,901:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/application?select=%2A%2C%20staff%28%2A%29&role_id=eq.2 "HTTP/1.1 200 OK"
2023-10-06 16:44:31,969:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/staff_skill?select=skill%28%2A%29&staff_id=eq.1 "HTTP/1.1 200 OK"
2023-10-06 16:44:32,016:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/role_skill?select=skill%28%2A%29&role_id=eq.2 "HTTP/1.1 200 OK"
2023-10-06 16:44:32,059:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/staff_skill?select=skill%28%2A%29&staff_id=eq.4 "HTTP/1.1 200 OK"
2023-10-06 16:44:32,117:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/role_skill?select=skill%28%2A%29&role_id=eq.2 "HTTP/1.1 200 OK"
2023-10-06 16:44:32,157:INFO - HTTP Request: GET https://wbsagjngbxrrzfktkvtt.supabase.co/rest/v1/staff_skill?select=skill%28%2A%29&staff_id=eq.1 "HTTP/1.1 200 OK"
2023-10-06 1

In [56]:
tes

[{'match_percentage': 50.0,
  'skill': [{'skill_id': 3,
    'skill_name': 'Scrum',
    'skill_desc': 'Use scrum methodology',
    'qualified': True},
   {'skill_id': 12,
    'skill_name': 'Express.js',
    'skill_desc': 'Node.js web application framework',
    'qualified': False}]},
 {'match_percentage': 0.0,
  'skill': [{'skill_id': 12,
    'skill_name': 'Express.js',
    'skill_desc': 'Node.js web application framework',
    'qualified': False},
   {'skill_id': 3,
    'skill_name': 'Scrum',
    'skill_desc': 'Use scrum methodology',
    'qualified': False}]},
 {'match_percentage': 50.0,
  'skill': [{'skill_id': 3,
    'skill_name': 'Scrum',
    'skill_desc': 'Use scrum methodology',
    'qualified': True},
   {'skill_id': 12,
    'skill_name': 'Express.js',
    'skill_desc': 'Node.js web application framework',
    'qualified': False}]},
 {'match_percentage': 50.0,
  'skill': [{'skill_id': 3,
    'skill_name': 'Scrum',
    'skill_desc': 'Use scrum methodology',
    'qualified': True}